Automating the @Geezus Metric 
The qb performance from the best college defense they played in their senior year

In [61]:
import cfbd as cfbd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [39]:
cfb_key = open('CFB\\CFBD_key.txt',"r").readline()


In [46]:
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = cfb_key
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class
api_instance = cfbd.PlayersApi(cfbd.ApiClient(configuration))
game_id = 56 # int | Game id filter (optional)
year = 2015 # int | Year/season filter for games (optional)
week = 8 # int | Week filter (optional)
season_type = 'regular' # str | Season type filter (regular or postseason) (optional) (default to regular)
team = 'team_example' # str | Team (optional)
home = 'home_example' # str | Home team filter (optional)
away = 'away_example' # str | Away team filter (optional)
conference = 'conference_example' # str | Conference abbreviation filter (optional)


Load in single player

find player on roster

In [103]:

selected_year = 2022
cfb_df = api_instance.player_search('Will Levis',position = 'QB',year = selected_year)

In [104]:
curr_team =  cfb_df[0].team


In [105]:
cfb_df[0]

{'first_name': 'Will',
 'height': 75,
 'hometown': 'Madison, CT',
 'id': 4361418,
 'jersey': 7,
 'last_name': 'Levis',
 'name': 'Will Levis',
 'position': 'QB',
 'team': 'Kentucky',
 'team_color': '#005DAA',
 'team_color_secondary': '#ffffff',
 'weight': 232}

In [124]:
team_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
stats_api_instance = cfbd.StatsApi(cfbd.ApiClient(configuration))


In [125]:
curr_team_list = team_api_instance.get_games(year = selected_year, team = curr_team)

In [126]:
player_opponent_name = []
player_opponent_id = []

for x in range(len(curr_team_list)):
    away_team = curr_team_list[x].away_team
    home_team = curr_team_list[x].home_team
    if away_team == curr_team:
        opp_id = curr_team_list[x].home_id
        if x == 0:
            curr_team_id = curr_team_list[x].away_id
    else:
        opp_id = curr_team_list[x].away_id
        if x == 0:
            curr_team_id = curr_team_list[x].home_id
    team_list = [away_team,home_team]
    team_list.remove(curr_team)

    player_opponent_name.append(team_list[0])
    player_opponent_id.append(opp_id)








In [127]:
print(player_opponent_id)
print(curr_team_id)

[193, 57, 2754, 2459, 145, 2579, 344, 2633, 142, 238, 61, 97]
96


In [128]:
player_opponent_name

['Miami (OH)',
 'Florida',
 'Youngstown State',
 'Northern Illinois',
 'Ole Miss',
 'South Carolina',
 'Mississippi State',
 'Tennessee',
 'Missouri',
 'Vanderbilt',
 'Georgia',
 'Louisville']

In [152]:
opponent_values = pd.DataFrame()
if True:

    i =0

    curr_opp_team = player_opponent_name[i]

    curr_opp_team_list = team_api_instance.get_games(year = selected_year, team = curr_opp_team)

    opp_team_stats = stats_api_instance.get_team_season_stats(year=selected_year, team=curr_opp_team, start_week=1, end_week=   16)

    curr_team_stats_dict = [x.to_dict() for x in opp_team_stats]
    for p in curr_team_stats_dict

    curr_team_stats_df = pd.DataFrame(curr_team_stats_dict)
    curr_team_stats_df = curr_team_stats_df.drop(columns = ['conference','team','season'])

    curr_wide = pd.pivot_table(curr_team_stats_df,columns = 'stat_name',fill_value = 0,aggfunc = sum)
    stats_of_interest = ['pass']
    curr_wide['row']


In [144]:
help(pd.pivot)

Help on function pivot in module pandas.core.reshape.pivot:

pivot(data: 'DataFrame', *, index: 'IndexLabel | None' = None, columns: 'IndexLabel | None' = None, values: 'IndexLabel | None' = None) -> 'DataFrame'
    Return reshaped DataFrame organized by given index / column values.
    
    Reshape data (produce a "pivot" table) based on column values. Uses
    unique values from specified `index` / `columns` to form axes of the
    resulting DataFrame. This function does not support data
    aggregation, multiple values will result in a MultiIndex in the
    columns. See the :ref:`User Guide <reshaping>` for more on reshaping.
    
    Parameters
    ----------
    data : DataFrame
    index : str or object or a list of str, optional
        Column to use to make new frame's index. If None, uses
        existing index.
    
        .. versionchanged:: 1.1.0
           Also accept list of index names.
    
    columns : str or object or a list of str
        Column to use to make new 

In [163]:

print(curr_wide)

stat_name   firstDowns  fourthDownConversions  fourthDowns  fumblesLost  \
stat_value         204                      7           17            6   

stat_name   fumblesRecovered  games  interceptionTDs  interceptionYards  \
stat_value                11     12                1                128   

stat_name   interceptions  kickReturnTDs  ...  puntReturns  rushingAttempts  \
stat_value              5              0  ...           23              446   

stat_name   rushingTDs  rushingYards  sacks  tacklesForLoss  \
stat_value          13          1721     26              55   

stat_name   thirdDownConversions  thirdDowns  totalYards  turnovers  
stat_value                    55         171        3711         11  

[1 rows x 32 columns]


In [123]:
len(stats_api_instance.get_stat_categories())

38

In [25]:
cfb_df[(cfb_df['first_name'] == 'Will') & (cfb_df['last_name'] == 'Levis') ]

,athlete_id,first_name,last_name,team,weight,height,jersey,year,position,home_city,home_state,home_country,home_latitude,home_longitude,home_county_fips,recruit_ids,headshot_url,season
5073,4361418,Will,Levis,Penn State,222.0,75.0,NaN,3.0,QB,Madison,CT,USA,41.2794282,-72.5983151,09009,[43559],https://a.espncdn.com/i/headshots/college-foot...,2020


In [14]:
cfb_df.columns

Index(['athlete_id', 'first_name', 'last_name', 'team', 'weight', 'height',
       'jersey', 'year', 'position', 'home_city', 'home_state', 'home_country',
       'home_latitude', 'home_longitude', 'home_county_fips', 'recruit_ids',
       'headshot_url', 'season'],
      dtype='object')